<a href="https://colab.research.google.com/github/sagar9926/EPAI_session16/blob/main/EPAI_session_16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Assignment 1 :

The files are:

- personal_info.csv - personal information such as name, gender, etc. (one row per person)

- vehicles.csv - what vehicle people own (one row per person)

- employment.csv - where a person is employed (one row per person)

- update_status.csv - when the person's data was created and last updated

Each file contains a key, SSN, which uniquely identifies a person.

This key is present in all four files.

You are guaranteed that the same SSN value is present in every file, and that it only appears once per file.

In addition, the files are all sorted by SSN, i.e. the SSN values appear in the same order in each file.

In [50]:
!git clone https://github.com/sagar9926/EPAI_session16.git

Cloning into 'EPAI_session16'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 18 (delta 4), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (18/18), done.


In [51]:
cd EPAI_session16

/content/EPAI_session16/EPAI_session16


In [52]:
!ls

employment.csv	       personal_info.csv  update_status.csv
EPAI_session_16.ipynb  README.md	  vehicles.csv


In [53]:
from collections import namedtuple
import datetime
import csv

In [54]:
def read_file(file_name):
  with open(file_name) as f:
    rows = csv.reader(f,delimiter = ',',quotechar = '"')
    yield from rows


In [55]:
data_type_dict = {
    "employment.csv" : [str,str,str,str],
    'personal_info.csv' : [str,str,str,str,str],
    'update_status.csv' : [str,datetime.datetime.strptime,datetime.datetime.strptime],
    'vehicles.csv' : [str,	str,	str,	str]

}
date_format = '%Y-%m-%dT%H:%M:%SZ'

In [56]:
class file_iter:
  def __init__(self,file_name):
    self.file_name = file_name
    self.data_type = data_type_dict[self.file_name]
    self.file = read_file(self.file_name)
    self.columns = " ".join(next(self.file))
    self.row_name_tuple = namedtuple((self.file_name[0].upper() + self.file_name[1:-4]),self.columns)
    self.row_index = 0
  
  def __iter__(self):
    for row in self.file:
      if self.file_name != 'update_status.csv':
        row = [type_(data) for type_ , data in zip(self.data_type,row)]
      else :
        row = [type_(data) if type_ == str else type_(data,date_format) for type_ , data in zip(self.data_type,row)]
      yield self.row_name_tuple(*row)



### Goal 1

Your first task is to create iterators for each of the four files that contained cleaned up data, of the correct type (e.g. string, int, date, etc), and represented by a named tuple.

For now these four iterators are just separate, independent iterators.

In [57]:
employment_iter = file_iter('employment.csv')
personal_info_iter = file_iter('personal_info.csv')
update_status_iter = file_iter('update_status.csv')
vehicles_iter = file_iter('vehicles.csv')

In [58]:
for i in update_status_iter:
  print(i)

Update_status(ssn='100-53-9824', last_updated=datetime.datetime(2017, 10, 7, 0, 14, 42), created=datetime.datetime(2016, 1, 24, 21, 19, 30))
Update_status(ssn='101-71-4702', last_updated=datetime.datetime(2017, 1, 23, 11, 23, 17), created=datetime.datetime(2016, 1, 27, 4, 32, 57))
Update_status(ssn='101-84-0356', last_updated=datetime.datetime(2017, 10, 4, 11, 21, 30), created=datetime.datetime(2016, 9, 21, 23, 4, 7))
Update_status(ssn='104-22-0928', last_updated=datetime.datetime(2017, 3, 28, 12, 38, 29), created=datetime.datetime(2016, 4, 15, 11, 37, 17))
Update_status(ssn='104-84-7144', last_updated=datetime.datetime(2018, 2, 19, 1, 34, 33), created=datetime.datetime(2016, 3, 15, 14, 7, 57))
Update_status(ssn='105-27-5541', last_updated=datetime.datetime(2017, 7, 24, 8, 58, 52), created=datetime.datetime(2016, 7, 23, 17, 58, 35))
Update_status(ssn='105-85-7486', last_updated=datetime.datetime(2018, 2, 14, 11, 32, 39), created=datetime.datetime(2016, 12, 15, 5, 46, 43))
Update_status